In [6]:
from keras.callbacks import ModelCheckpoint
from network.waifu2x import waifu2x

import cv2
import numpy as np
from scipy import misc

from utility.h5data import h5DataRead

import math
import time

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [7]:
EPOCHS = 4
BATCH_SIZE = 128

SIZE_X = 160
SIZE_Y = 120
SCALE = 4

FILENAME_TRAIN = "yayoi_waifu2x_dataTrain_" + str(SCALE)  + ".h5"
FILENAME_VALIDATE = "yayoi_waifu2x_dataValidate_" + str(SCALE) + ".h5"
FILENAME_CHECKPOINT = "yayoi_waifu2x_dataCheckPoint_" + str(SCALE) + ".h5"

In [8]:
# dnn configuration for waifu2x
dnn = waifu2x()
#dnn.layers = 7
#dnn.filters = [32, 32, 64, 64, 128, 128, 1]
#dnn.kernelSize = [(3,3), (3,3), (3,3), (3,3), (3,3), (3,3), (3,3)]
dnn.inputShape = (SIZE_Y, SIZE_X, 1)
dnn.in_train = True

In [9]:
tick1 = cv2.getTickCount()

In [10]:
model = dnn.network()
print(model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

checkpoint = ModelCheckpoint(FILENAME_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')

callbacks_list = [checkpoint]

model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                callbacks=callbacks_list, shuffle=True, epochs=EPOCHS, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 120, 160, 32)      320       
_________________________________________________________________
batch_normalization_7 (Batch (None, 120, 160, 32)      128       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 120, 160, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 120, 160, 32)      9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 120, 160, 32)      128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 120, 160, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 120, 160, 64)      18496     
__________

ValueError: Error when checking input: expected conv2d_8_input to have 4 dimensions, but got array with shape (80, 1, 3, 120, 160)

In [ ]:
score = model.evaluate(val_data, val_label, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
fileName = 'yayoi_waifu2x_' + str(SCALE) + "_" + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
model.save_weights(fileName)
print(fileName + ' saved')

In [ ]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Training processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Training processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Training processed time: " + str(tick) + " ms")

In [ ]:
tick1 = cv2.getTickCount()

IMG_NAME = "./imageTrain/yayoi_first_030.png"
IMG_OPENCV_NAME = "opencv.png"
IMG_DNN_NAME = "waifu2x.png"

INTERPOLATION = cv2.INTER_CUBIC

In [ ]:
dnn.in_train = False
model = dnn.network()
model.load_weights(fileName)
print(model.summary())

In [ ]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
shape = img.shape

img = cv2.resize(img, (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
img = cv2.resize(img, shape[1], shape[0], INTERPOLATION)

cv2.imwrite(IMG_OPENCV_NAME, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

In [ ]:
Y = numpy.zeros((1, img.shape[0], img.shape[1], 3), dtype=float)
Y[0, :, :, :] = img.astype(float) / 255.

pre = model.predict(Y, batch_size=1) * 255.
pre[pre[:] > 255] = 255
pre[pre[:] < 0] = 0

pre = pre.astype(numpy.uint8)
cv2.imwrite(IMG_DNN_NAME, pre, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

In [ ]:
# PSNR calculation
im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
im2 = cv2.imread(IMG_OPENCV_NAME, cv2.IMREAD_COLOR)
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
im3 = cv2.imread(IMG_DNN_NAME, cv2.IMREAD_COLOR)
im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)

print("opencv:")
print(cv2.PSNR(im1, im2))
print("waifu2x:")
print(cv2.PSNR(im1, im3))

In [ ]:
# display image
pltShow = False

if pltShow:    
    plt.figure(num='comparison',figsize=(16,16))

    plt.subplot(1,3,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(1,3,2)
    plt.title('OpenCV')
    plt.imshow(im2)

    plt.subplot(1,3,3)
    plt.title('waifu2x')
    plt.imshow(im3)

In [ ]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("Inference processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("Inference processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("Inference processed time: " + str(tick) + " ms")